# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
#instalar o webdriver compatível com o meu navegador
#pip install webdriver-manager
#from webdriver_manager.edge import EdgeDriverManager
# selenium 3
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager

driver = webdriver.Edge(EdgeChromiumDriverManager().install())
# selenium 4
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager

driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))

In [27]:
#pip install selenium
from selenium import webdriver #Em vez de importar a bib inteira, só pegamos o webdriver
from selenium.webdriver.common.keys import Keys
#pip install cx-Freeze #para transformar em um exe.


# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

browser = webdriver.Edge()
# Passo 1: Pegar a cotação do dólar
# entrar no google
browser.get("https://www.google.com/")
browser.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação dólar", Keys.ENTER)
#browser.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[4]/center/input[1]").click()
dollarValue = browser.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(dollarValue)

# Passo 2: Pegar a cotação do euro
browser.get("https://www.google.com/")
browser.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação euro", Keys.ENTER)
euroValue = browser.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(euroValue)

# Passo 3: Pegar a cotação do ouro
browser.get("https://www.melhorcambio.com/ouro-hoje")
goldValue = browser.find_element("xpath", '//*[@id="comercial"]').get_attribute("value")
goldValue =  goldValue.replace(",", ".")
print(goldValue)


with open(r"C:\Users\felip\OneDrive\Documentos\cotas.txt", "w") as file:
    file.write(f"O preço do dólar é: {dollarValue}\nO preço do ouro é: {goldValue}\nO preço do euro é: {euroValue}")
    
browser.quit()




ModuleNotFoundError: No module named 'webdriver_manager'

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [24]:
# Passo 4: Importar a base de dados e Atualizar a base
import pandas as pd

table = pd.read_excel("Produtos.xlsx")
print(table)

         Produtos  Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar        5          4999.95    1.40   
1   Carro Renault         4500.00   Euro        6         27000.00    2.00   
2   Notebook Dell          899.99  Dólar        5          4499.95    1.70   
3          IPhone          799.00  Dólar        5          3995.00    1.70   
4      Carro Fiat         3000.00   Euro        6         18000.00    1.90   
5  Celular Xiaomi          480.48  Dólar        5          2402.40    2.00   
6        Joia 20g           20.00   Ouro      350          7000.00    1.15   

   Preço de Venda  
0        6999.930  
1       54000.000  
2        7649.915  
3        6791.500  
4       34200.000  
5        4804.800  
6        8050.000  


- Atualizando os preços e o cálculo do Preço Final

In [25]:
import pandas as pd

table = pd.read_excel("Produtos.xlsx")
# Passo 5: Recalcular os preços
# atualizar cotação
# cotação dólar
table.loc[table["Moeda"] == "Dólar", "Cotação"] = float(dollarValue)
table.loc[table["Moeda"] == "Euro", "Cotação"] = float(euroValue)
table.loc[table["Moeda"] == "Ouro", "Cotação"] = float(goldValue)
# preço de compra = cotação * preço original
table["Preço de Compra"] = table["Cotação"] * table["Preço Original"]

# preço de venda = preço de compra * margem
#table["Preço de Venda"] = table["Preço de Compra"] * table["Margem"]
# formatado
table["Preço de Venda"] = table["Preço de Venda"].map("R${:.2f}".format)
print(table)


         Produtos  Preço Original  Moeda     Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar    5.087700      5087.649123    1.40   
1   Carro Renault         4500.00   Euro    5.516847     24825.813728    2.00   
2   Notebook Dell          899.99  Dólar    5.087700      4578.879123    1.70   
3          IPhone          799.00  Dólar    5.087700      4065.072300    1.70   
4      Carro Fiat         3000.00   Euro    5.516847     16550.542485    1.90   
5  Celular Xiaomi          480.48  Dólar    5.087700      2444.538096    2.00   
6        Joia 20g           20.00   Ouro  315.120000      6302.400000    1.15   

  Preço de Venda  
0      R$6999.93  
1     R$54000.00  
2      R$7649.91  
3      R$6791.50  
4     R$34200.00  
5      R$4804.80  
6      R$8050.00  


### Agora vamos exportar a nova base de preços atualizada

In [26]:
# Passo 6: Exportar a base atualizada
table.to_excel("ProdutosNew.xlsx", index = False)
